## 用户认证 - 由self.perform_authentication实现
    
    1.遍历当前视图配置的用户认证类所组成列表
    2.遍历时执行认证类的authenticate方法。其原理为：
        倘若验证不通过，则引发异常，框架会直接返回response，自动跳过后续认证类及视图函数
        倘若验证通过，列表最后一个验证类的authenticate方法应返回一个由用户对象、认证凭证组成的二元组(self.user, selff.auth)，其他验证类返回None，继续往后执行其他验证类。

#### 1. 获取用户认证类组成的列表

    要完成遍历用户认证列表的操作，则必须先了解drf如何获取到该列表：
    在对request做进一步加工时(实例化Request)，调用self.get_authenticator方法（此处的self指向classView），并保存在Request.authenticators属性中

#### self.get_authenticator源码

In [ ]:
    def get_authenticators(self):
        """
        Instantiates and returns the list of authenticators that this view can use.
        """
        return [auth() for auth in self.authentication_classes]
    
    
    # APIView中含有self.authentication_classes属性
    self.authentication_classes = api_settings.DEFAULT_AUTHENTICATION_CLASSES

#### 2.遍历用户认证类
    

In [ ]:
    def perform_authentication(self, request):
        """
        Perform authentication on the incoming request.

        Note that if you override this and simply 'pass', then authentication
        will instead be performed lazily, the first time either
        `request.user` or `request.auth` is accessed.
        """
        request.user
        
    # request.user 是property属性
    
    @property
    def user(self):
        """
        Returns the user associated with the current request, as authenticated
        by the authentication classes provided to the request.
        """
        if not hasattr(self, '_user'):
            with wrap_attributeerrors():
                self._authenticate()
        return self._user   # 目前来看，这个返回值似乎没用
    
    
    # 在该方法中完成遍历
    def _authenticate(self):
    """
    Attempt to authenticate the request using each authentication instance
    in turn.
    """
    for authenticator in self.authenticators:
        try:
            user_auth_tuple = authenticator.authenticate(self)
        except exceptions.APIException:
            self._not_authenticated()
            raise

        if user_auth_tuple is not None:
            self._authenticator = authenticator
            self.user, self.auth = user_auth_tuple
            return

    self._not_authenticated()
    
    

### 3.认证demo

#### 3.1局部配置
    
    在获取认证列表时，通过self.authentication_classes属性进行获取，则可以在类视图中定义一个authentication_classes属性，该属性为列表，在其中添加自定义的认证类。

In [ ]:
from rest_framework.authentication import BaseAuthentication
from rest_framework.exceptions import AuthenticationFailed
from rest_framework.view import APIView

class UserAuthentication(BaseAuthentication):
    """验证用户登录"""
    
    def authenticator(self, request):
        
        user = getattr(request._request, 'user', None)
        
        if not user and not user.is_active:
            raise AuthenticationFailed("未通过验证！")
        # 返回二元组，最终赋值情况为：request.user = user, self.auth = None
        return user, None
    
    
class MyView(APIView):
    
    authentication_classes = [UserAuthentication]
    
    def get(self, request):
        pass

#### 3.2 全局配置
    
    倘若需要对所有类视图进行验证，则可以在settings.py中进行全局验证。
    
##### 值得一提的是，drf会先在视图中寻找认证列表，然后再从配置文件中加载。
    基于这一点，可以对个别不需要进行全局认证的视图进行更加精细的控制
    

In [ ]:
# settings.py

REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': ['api.utils.auth.UserAuthentication',]
}

### 自定义认证类
    
    主要为两点：
    1.继承rest_framework.authentication.BaseAuthentication
    2.重写authenticate(self, request)方法。
        如果有多个认证类，认证成功时，列表前面的认证类应返回None,最后一个认证类返回一个二元组（用户对象，认证凭证）。认证凭证可以为token, session_id，也可以直接为None。
        如果认证失败，则抛出rest_framework.exceptions.AuthenticationFailed异常